## Movie Recommender Systems

The rapid growth of data collection has led to a new era of information. Data is being used to create more efficient systems and this is where Recommendation Systems come into play. They are used to predict the rating or preference that a user would give to an item. Almost every major tech company has applied them in some form or the other: Amazon uses it to suggest products to customers, YouTube uses it to decide which video to play next on autoplay, and Facebook uses it to recommend pages to like and people to follow. Moreover, companies like Netflix and Spotify depend highly on the effectiveness of their recommendation engines for their business and success.

In this notebook, I will try to predict the score or the rating of a movie. We know that the score of a movie depends on various factors like the genre, or the actor working in the film and mainly the director of the film. Considering all such factors, I will try to build a simple score predictor for this dataset.

General description and data are available on [Kaggle](https://www.kaggle.com/tmdb/tmdb-movie-metadata).

1. [Load Dataset](#1)

2. [Demographic Filtering](#2)

3. [Content Based Filtering](#3)

4. [Collaborative Filtering](#4)

## <a name="1">Load Dataset</a>

In [ ]:
import pandas as pd

movies=pd.read_csv("data/tmdb_5000_movies.csv")
credits=pd.read_csv("data/tmdb_5000_credits.csv")

columns_names=movies.columns.tolist()
print("Columns names: \n{}".format(columns_names))
print(movies.shape)
movies.head()

We can see that genres,keywords,production_companies,production_countries,spoken_languages are in the json format. Similarly in the other csv file, cast and crew are in the json format. Now lets convert these columns into a format that can be easily read and interpreted. We will convert them into strings and later convert them into lists for easier interpretation.

In [ ]:
import json

# Converting the json into strings
movies["genres"]=movies["genres"].apply(json.loads)
for index,i in zip(movies.index,movies["genres"]):
    list1=[]
    for j in range(len(i)):
        list1.append((i[j]["name"])) # the key "name" contains the name of the genre
    movies.loc[index,"genres"]=str(list1)

movies["keywords"]=movies["keywords"].apply(json.loads)
for index,i in zip(movies.index,movies["keywords"]):
    list1=[]
    for j in range(len(i)):
        list1.append((i[j]["name"]))
    movies.loc[index,"keywords"]=str(list1)

movies["production_companies"]=movies["production_companies"].apply(json.loads)
for index,i in zip(movies.index,movies["production_companies"]):
    list1=[]
    for j in range(len(i)):
        list1.append((i[j]["name"]))
    movies.loc[index,"production_companies"]=str(list1)

movies["production_countries"]=movies["production_countries"].apply(json.loads)
for index,i in zip(movies.index,movies["production_countries"]):
    list1=[]
    for j in range(len(i)):
        list1.append((i[j]["name"]))
    movies.loc[index,"production_countries"]=str(list1)
    
credits["cast"]=credits["cast"].apply(json.loads)
for index,i in zip(credits.index,credits["cast"]):
    list1=[]
    for j in range(len(i)):
        list1.append((i[j]["name"]))
    credits.loc[index,"cast"]=str(list1)

credits["crew"]=credits["crew"].apply(json.loads)
def director(x):
    for i in x:
        if i["job"]=="Director":
            return i["name"]
credits["crew"]=credits["crew"].apply(director)
credits.rename(columns={"crew":"director"},inplace=True)

movies.head()

Let's join the two dataset on the 'id' column:

In [ ]:
credits.columns=["id","title_y","cast","crew"]
movies=movies.merge(credits,on="id")

movies.head()

## <a name="2">Demographic Filtering</a>

Before getting started with this:
- we need a metric to score or rate movie
- Calculate the score for every movie
- Sort the scores and recommend the best rated movie to the users.

We can use the average ratings of the movie as the score but using this won't be fair enough since a movie with 8.9 average rating and only 3 votes cannot be considered better than the movie with 7.8 as as average rating but 40 votes. So, let's use [IMDB's weighted rating (wr)](https://math.stackexchange.com/questions/169032/understanding-the-imdb-weighted-rating-function-for-usage-on-my-own-website)

where,
- v is the number of votes for the movie;
- m is the minimum votes required to be listed in the chart;
- R is the average rating of the movie; And
- C is the mean vote across the whole report

We already have v (vote_count) and R (vote_average) and C can be calculated as

In [ ]:
C=movies["vote_average"].mean()
C

So, the mean rating for all the movies is approx 6 on a scale of 10.The next step is to determine an appropriate value for m, the minimum votes required to be listed in the chart. We will use 90th percentile as our cutoff. In other words, for a movie to feature in the charts, it must have more votes than at least 90% of the movies in the list.

In [ ]:
m=movies["vote_count"].quantile(0.9)
print(m)

# Let's filter out the movies that qualify for the chart
q_movies=movies.copy().loc[movies["vote_count"]>=m]
q_movies.shape

We see that there are 481 movies which qualify to be in this list. Now, we need to calculate our metric for each qualified movie. To do this, we will define a function, weighted_rating() and define a new feature score, of which we'll calculate the value by applying this function to our DataFrame of qualified movies:

In [ ]:
def weighted_rating(x,m=m,C=C):
    v=x["vote_count"]
    R=x["vote_average"]
    # Calculation based on the IMDB formula
    return (v/(v+m)*R)+(m/(m+v)*C)

# Define a new feature 'score' and calculate its value
q_movies["score"]=q_movies.apply(weighted_rating,axis=1)

Finally, let's sort the DataFrame based on the score feature and output the title, vote count, vote average and weighted rating or score of the top 10 movies.

In [ ]:
# Sort movies based on score calculated above
q_movies=q_movies.sort_values("score",ascending=False)

q_movies[["title","vote_count","vote_average","score"]].head(10)

We have made our first(though very basic) recommender. Under the Trending Now tab of these systems we find movies that are very popular and they can just be obtained by sorting the dataset by the popularity column.

In [ ]:
pop=movies.sort_values("popularity",ascending=False)

import matplotlib.pyplot as plt

plt.figure(figsize=(12,4))
plt.barh(pop["title"].head(6),pop["popularity"].head(6),align="center")
plt.gca().invert_yaxis()
plt.xlabel("Popularity")
plt.title("Popular Movies")
plt.show()

Now something to keep in mind is that these demographic recommender provide a general chart of recommended movies to all the users. They are not sensitive to the interests and tastes of a particular user. This is when we move on to a more refined system- Content Based Filtering.

## <a name="3">Content Based Filtering</a>

In this recommender system the content of the movie (overview, cast, crew, keyword, tagline etc) is used to find its similarity with other movies. Then the movies that are most likely to be similar are recommended.

**Plot description based Recommender**

We will compute pairwise similarity scores for all movies based on their plot descriptions and recommend movies based on that similarity score. The plot description is given in the overview feature of our dataset. Let's take a look at the data.

In [ ]:
movies["overview"].head()

Now we'll compute Term Frequency-Inverse Document Frequency (TF-IDF) vectors for each overview.

Term frequency , it is the relative frequency of a word in a document and is given as (term instances/total instances). Inverse Document Frequency is the relative count of documents containing the term is given as log(number of documents/documents with term) The overall importance of each word to the documents in which they appear is equal to TF * IDF

This will give us a matrix where each column represents a word in the overview vocabulary (all the words that appear in at least one document) and each column represents a movie, as before. This is done to reduce the importance of words that occur frequently in plot overviews and therefore, their significance in computing the final similarity score.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Define a TF-IDF Vectorizer Object
# Remove all english stop words such as 'the', 'a'
tfidf=TfidfVectorizer(stop_words="english")

# Replace NaN with an empty string
movies["overview"]=movies["overview"].fillna("")

# Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix=tfidf.fit_transform(movies["overview"])

tfidf_matrix.shape

We see that over 20,000 different words were used to describe the 4800 movies in our dataset.

With this matrix in hand, we can now compute a similarity score. There are several candidates for this; such as the euclidean, the Pearson and the [cosine similarity score](https://en.wikipedia.org/wiki/Cosine_similarity). There is no right answer to which score is the best. Different scores work well in different scenarios and it is often a good idea to experiment with different metrics.

We will be using the cosine similarity to calculate a numeric quantity that denotes the similarity between two movies. We use the cosine similarity score since it is independent of magnitude and is relatively easy and fast to calculate.

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim=linear_kernel(tfidf_matrix,tfidf_matrix)

We are going to define a function that takes in a movie title as an input and outputs a list of the 10 most similar movies. Firstly, for this, we need a reverse mapping of movie titles and DataFrame indices. In other words, we need a mechanism to identify the index of a movie in our metadata DataFrame, given its title.

In [ ]:
# Construct a reverse map of indices and movie titles
indices=pd.Series(movies.index,index=movies["title"]).drop_duplicates()

We are now in a good position to define our recommendation function. These are the following steps we'll follow :
- Get the index of the movie given its title.
- Get the list of cosine similarity scores for that particular movie with all movies. Convert it into a list of tuples where the first element is its position and the second is the similarity score.
- Sort the aforementioned list of tuples based on the similarity scores; that is, the second element.
- Get the top 10 elements of this list. Ignore the first element as it refers to self (the movie most similar to a particular movie is the movie itself).
- Return the titles corresponding to the indices of the top elements.

In [ ]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title,cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx=indices[title]
    
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores=list(enumerate(cosine_sim[idx]))
    
    # Sort the movies based on the similarity scores
    sim_scores=sorted(sim_scores,key=lambda x: x[1],reverse=True)
    
    # Get the scores of the 10 most similar movies
    sim_scores=sim_scores[1:11]
    
    # Get the movie indices
    movie_indices=[i[0] for i in sim_scores]
    
    # Return the top 10 most similar movies
    return movies["title"].iloc[movie_indices]

In [ ]:
get_recommendations("The Dark Knight Rises")

While our system has done a decent job of finding movies with similar plot descriptions, the quality of recommendations is not that great. "The Dark Knight Rises" returns all Batman movies while it is more likely that the people who liked that movie are more inclined to enjoy other Christopher Nolan movies. This is something that cannot be captured by the present system.

**Credits, Genres and Keywords Based Recommender**

It goes without saying that the quality of our recommender would be increased with the usage of better metadata. That is exactly what we are going to do in this section. We are going to build a recommender based on the following metadata: the 3 top actors, the director, related genres and the movie plot keywords.

From the cast, crew and keywords features, we need to extract the three most important actors, the director and the keywords associated with that movie. Right now, our data is present in the form of "stringified" lists , we need to convert it into a safe and usable structure

In [ ]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features=["cast","crew","keywords","genres"]
for feature in features:
    movies[feature]=movies[feature].apply(literal_eval)

Next, we'll write functions that will help us to extract the required information from each feature.

In [ ]:
import numpy as np

# Get the director's name from the crew feature. If director is not listed, return NaN
def get_director(x):
    for i in x:
        if i["job"]=="Director":
            return i["name"]
    return np.nan

In [ ]:
# Returns the list top 3 elements or entire list; whichever is more
def get_list(x):
    if isinstance(x,list):
        names=[i["name"] for i in x]
        # Check if more than 3 elements exist. If yes, return only first three. If no, return entire list
        if len(names)>3:
            names=names[:3]
        return names
    
    # Return empty list in case of missing/malformed data
    return []

In [ ]:
# Define new director, cast, genres and keywords features that are in a suitable form
movies["director"]=movies["crew"].apply(get_director)

features=["cast","keywords","genres"]
for feature in features:
    movies[feature]=movies[feature].apply(get_list)
    
# Print the new features of the first 3 films
movies[["title","cast","director","keywords","genres"]].head(3)

The next step would be to convert the names and keyword instances into lowercase and strip all the spaces between them. This is done so that our vectorizer doesn't count the Johnny of "Johnny Depp" and "Johnny Galecki" as the same.

In [ ]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x,list):
        return [str.lower(i.replace(" ","")) for i in x]
    else:
        # Check if director exists. If not, return empty string
        if isinstance(x,str):
            return str.lower(x.replace(" ",""))
        else:
            return ""

In [ ]:
# Apply clean_data function to your features
features=["cast","keywords","director","genres"]

for feature in features:
    movies[feature]=movies[feature].apply(clean_data)

We are now in a position to create our "metadata soup", which is a string that contains all the metadata that we want to feed to our vectorizer (namely actors, director and keywords).

In [ ]:
def create_soup(x):
    return " ".join(x["keywords"])+" "+" ".join(x["cast"])+" "+x["director"]+" "+" ".join(x["genres"])
movies["soup"]=movies.apply(create_soup,axis=1)

The next steps are the same as what we did with our plot description based recommender. One important difference is that we use the CountVectorizer() instead of TF-IDF. This is because we do not want to down-weight the presence of an actor/director if he or she has acted or directed in relatively more movies. It doesn't make much intuitive sense.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count=CountVectorizer(stop_words="english")
count_matrix=count.fit_transform(movies["soup"])

In [ ]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2=cosine_similarity(count_matrix,count_matrix)

In [ ]:
# Reset index of our main DataFrame and construct reverse mapping as before
movies=movies.reset_index()
indices=pd.Series(movies.index,index=movies["title"])

We can now reuse our get_recommendations() function by passing in the new cosine_sim2 matrix as your second argument.

In [ ]:
get_recommendations("The Dark Knight Rises",cosine_sim2)

We see that our recommender has been successful in capturing more information due to more metadata and has given us (arguably) better recommendations. It is more likely that Marvels or DC comics fans will like the movies of the same production house. Therefore, to our features above we can add production_company . We can also increase the weight of the director , by adding the feature multiple times in the soup.

## <a name="4">Collaborative Filtering</a>

Our content based engine suffers from some severe limitations. It is only capable of suggesting movies which are close to a certain movie. That is, it is not capable of capturing tastes and providing recommendations across genres.

Also, the engine that we built is not really personal in that it doesn't capture the personal tastes and biases of a user. Anyone querying our engine for recommendations based on a movie will receive the same recommendations for that movie, regardless of who she/he is.

Therefore, in this section, we will use a technique called Collaborative Filtering to make recommendations to Movie Watchers. It is basically of two types:
- User based filtering: These systems recommend products to a user that similar users have liked. For measuring the similarity between two users we can either use pearson correlation or cosine similarity. Although computing user-based CF is very simple, it suffers from several problems. One main issue is that users preference can change over time. It indicates that precomputing the matrix based on their neighboring users may lead to bad performance. To tackle this problem, we can apply item-based CF.
- Item Based Collaborative Filtering: Instead of measuring the similarity between users, the item-based CF recommends items based on their similarity with the items that the target user rated. Likewise, the similarity can be computed with Pearson Correlation or Cosine Similarity. It successfully avoids the problem posed by dynamic user preference as item-based CF is more static. However, several problems remain for this method. First, the main issue is scalability. The computation grows with both the customer and the product. The worst case complexity is O(mn) with m users and n items. In addition, sparsity is another concern.

**Single Value Decomposition**

One way to handle the scalability and sparsity issue created by CF is to leverage a latent factor model to capture the similarity between users and items. Essentially, we want to turn the recommendation problem into an optimization problem. We can view it as how good we are in predicting the rating for items given a user. One common metric is Root Mean Square Error (RMSE). The lower the RMSE, the better the performance.

Now talking about latent factor you might be wondering what is it ?It is a broad idea which describes a property or concept that a user or an item have. For instance, for music, latent factor can refer to the genre that the music belongs to. SVD decreases the dimension of the utility matrix by extracting its latent factors. Essentially, we map each user and each item into a latent space with dimension r. Therefore, it helps us better understand the relationship between users and items as they become directly comparable.

Let's see how to implement this. Since the dataset we used before did not have userId (which is necessary for collaborative filtering) let's load another dataset. We'll be using the Surprise library to implement SVD.

In [ ]:
from surprise import Reader,Dataset,SVD

reader=Reader()
ratings=pd.read_csv("data/ratings_small.csv")

Note that in this dataset movies are rated on a scale of 5 unlike the earlier one.

In [ ]:
data=Dataset.load_from_df(ratings[["userId","movieId","rating"]],reader)
# data.split(n_folds=5)

In [ ]:
svd=SVD()

from surprise.model_selection import cross_validate

cross_validate(svd,data,measures=['RMSE', 'MAE'],cv=5,verbose=True)

We get a mean Root Mean Square Error of 0.89 approx which is more than good enough for our case. Let's now train on our dataset and arrive at predictions.

In [ ]:
train=data.build_full_trainset()
svd.fit(train)

Let's pick user with user Id 1 and check the ratings she/he has given.

In [ ]:
ratings[ratings["userId"]==1]

In [ ]:
svd.predict(1,302,3)

For movie with ID 302, we get an estimated prediction of 2.618. One startling feature of this recommender system is that it doesn't care what the movie is (or what it contains). It works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have predicted the movie.